In [1]:
# CatBoost

In [2]:
# IMPORT LIBRARY PACKAGES
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# LOAD GIVEN DATASET
data=pd.read_csv('hospital.csv')

In [5]:
df=data.dropna()

In [6]:
df.columns

Index(['outcome', 'age', 'BMI', 'heart rate', 'Systolic blood pressure',
       'Diastolic blood pressure', 'Respiratory rate', 'temperature', 'SP O2',
       'Urine output', 'glucose', 'PH'],
      dtype='object')

In [7]:
df.head()

,outcome,age,BMI,heart rate,Systolic blood pressure,Diastolic blood pressure,Respiratory rate,temperature,SP O2,Urine output,glucose,PH
0,0.0,72,37.588179,68.837838,155.866667,68.333333,16.621622,36.714286,98.394737,2155.0,114.636364,7.230
2,0.0,83,26.572634,72.318182,135.333333,61.375000,23.640000,36.453704,95.291667,2425.0,149.000000,7.268
3,0.0,43,83.264629,94.500000,126.400000,73.200000,21.857143,36.287037,93.846154,8760.0,128.250000,7.370
4,0.0,75,31.824842,67.920000,156.560000,58.120000,21.360000,36.761905,99.280000,4455.0,145.750000,7.250
5,0.0,76,24.262293,74.181818,118.100000,52.950000,20.545455,35.266667,96.818182,1840.0,98.333333,7.310


In [8]:
df=df.rename({'heart rate':'heart_rate','Systolic blood pressure':'Systolic_blood_pressure',
              'Diastolic blood pressure':'Diastolic_blood_pressure','Respiratory rate':'Respiratory_rate',
             'SP O2':'SP_O2','Urine output':'Urine_output'},axis=1)

In [9]:
df.columns

Index(['outcome', 'age', 'BMI', 'heart_rate', 'Systolic_blood_pressure',
       'Diastolic_blood_pressure', 'Respiratory_rate', 'temperature', 'SP_O2',
       'Urine_output', 'glucose', 'PH'],
      dtype='object')

In [10]:
from sklearn.preprocessing import LabelEncoder
encode=['outcome','age','BMI','heart_rate','Systolic_blood_pressure','Diastolic_blood_pressure','Respiratory_rate',
        'temperature','SP_O2','Urine_output','glucose','PH']
label=LabelEncoder()
for i in encode:
    df[i]=label.fit_transform(df[i].astype(int))

In [11]:
df.head()

,outcome,age,BMI,heart_rate,Systolic_blood_pressure,Diastolic_blood_pressure,Respiratory_rate,temperature,SP_O2,Urine_output,glucose,PH
0,0,38,24,18,69,35,5,3,12,347,41,0
2,0,49,13,22,50,28,12,3,9,388,76,0
3,0,9,53,44,41,40,10,3,7,553,55,0
4,0,41,18,17,70,25,10,3,13,514,72,0
5,0,42,11,24,33,19,9,2,10,304,25,0


In [12]:
# preprocessing, split test and dataset,split response variable
X=df.drop(labels='outcome',axis=1)
# Response variable
y=df.loc[:,'outcome']

In [24]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
X=scaler.fit_transform(X,y)

In [25]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros=RandomOverSampler(random_state=42)
x_ros,y_ros=ros.fit_resample(X,y)
print('OUR DATASET COUNT         :',Counter(y))
print('OVER SAMPLING DATA COUNT  :',Counter(y_ros))

OUR DATASET COUNT         : Counter({0: 608, 1: 101})
OVER SAMPLING DATA COUNT  : Counter({0: 608, 1: 608})


In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x_ros,y_ros,test_size=0.30,random_state=42,stratify=y_ros)
print('Number of training dataset:',len(X_train))
print('Number of test dataset    :',len(X_test))
print('Total Number of Dataset   :',len(X_train)+len(X_test))

Number of training dataset: 851
Number of test dataset    : 365
Total Number of Dataset   : 1216


In [27]:
# implementing CatBoost Logo
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,plot_confusion_matrix

In [28]:
# Training
cboost=CatBoostClassifier(verbose=0)
cboost.fit(X_train,y_train)
predicted=cboost.predict(X_test)

In [29]:
# Finding Accuracy
accuracy=accuracy_score(y_test,predicted)
print('Accuracy of CatBoost',accuracy*100)

Accuracy of CatBoost 93.97260273972603


In [30]:
# Finding Classification Report
cr=classification_report(y_test,predicted)
print('Classification report\n\n',cr)

Classification report

               precision    recall  f1-score   support

           0       0.98      0.90      0.94       183
           1       0.91      0.98      0.94       182

    accuracy                           0.94       365
   macro avg       0.94      0.94      0.94       365
weighted avg       0.94      0.94      0.94       365



In [31]:
# Finding Confusion Matrix
cm=confusion_matrix(y_test,predicted)
print('Confusion Matrix\n\n',cm)

Confusion Matrix

 [[165  18]
 [  4 178]]


In [32]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(figsize=(10,10))
plot_confusion_matrix(cboost,X_test,y_test,ax=ax)
plt.title('Confusion Matrix of CatBoost')
plt.show()

In [33]:
df2=pd.DataFrame()
df2['y_test']=y_test
df2['predicted']=predicted
df2.reset_index(inplace=True)
plt.figure(figsize=(20,5))
plt.plot(df2['predicted'][:100],marker='x',linestyle='dashed',color='red')
plt.plot(df2['y_test'][:100],marker='o',linestyle='dashed',color='purple')
plt.show()

In [22]:
# Creating File

In [23]:
import joblib
joblib.dump(cboost,'catboost.pkl')

['catboost.pkl']